Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 6.6 ms, sys: 993 µs, total: 7.59 ms
Wall time: 14.1 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.1.0.4/6933/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 6 Memory: 7.29 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 280 ms, sys: 25.1 ms, total: 305 ms
Wall time: 2.07 s


[1.0108208156172729,
 1.4028079381908338,
 1.3680389559198711,
 1.5606139629607312,
 1.8851528265197697,
 1.1382623752603254,
 1.7934055016997283,
 1.0123766407401984,
 1.4782979060403183,
 1.6903411676739015,
 1.0339667575887992,
 1.1752848907922653,
 1.1199868510277393,
 1.436194388626657,
 1.4619950521158218,
 1.5597942769407565,
 1.3267173001593195,
 1.4621661930386503,
 1.8090596077542926,
 1.961455575499399,
 1.754216339480485,
 1.9144561737772818,
 1.9046623579825175,
 1.8699094238212481,
 1.2743643422983577,
 1.3989353640916415,
 1.403607109747145,
 1.0040463818791014,
 1.494709826489447,
 1.950032848033293,
 1.4548324956210976,
 1.2565447804998922,
 1.0081768656447234,
 1.8543879317237177,
 1.9223064619607926,
 1.3377923377067167,
 1.0745248479955496,
 1.3912466125569627,
 1.45770450783848,
 1.788503848991324,
 1.091231562611234,
 1.1229814023427125,
 1.950237030540766,
 1.2422520294833113,
 1.1118872874653507,
 1.2376542515182556,
 1.4568873993193951,
 1.1818302980183195,
 1.

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.8407762630334292,
 1.8368455636999474,
 1.9017815160423266,
 1.7911599568134144,
 1.2993374595451421,
 1.5635461421006676,
 1.7183337388534348,
 1.976519437172708,
 1.5332907058620315,
 1.2378230794134935,
 1.6420221680900355,
 1.062918228585508,
 1.170305145883864,
 1.0411643055711277,
 1.6020679454697866,
 1.4442425461800723,
 1.3517053400050874,
 1.9281501272762553,
 1.2072130359334143,
 1.7214295546482763,
 1.88750960288698,
 1.3133382070274955,
 1.2638980255000911,
 1.1630506844724384,
 1.3397257302478223,
 1.172299621739958,
 1.5176249993462754,
 1.554173332684881,
 1.976728200488397,
 1.0814757962699175,
 1.3248742275606293,
 1.0990409937566172,
 1.3934894321846674,
 1.6921097492127735,
 1.9826223050370497,
 1.3419007565002443,
 1.4098966459120053,
 1.433067559230564,
 1.85088451608993,
 1.5928544006147392,
 1.9122479215788086,
 1.3317252584153387,
 1.8558007414016773,
 1.1954663089555528,
 1.280887411837444,
 1.8390722051169996,
 1.323308223425786,
 1.42407568229968,
 1.4093

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 202 ms, sys: 18.3 ms, total: 221 ms
Wall time: 2.4 s
